# Lab 4A: Hosted Agents with Azure Developer CLI

Deploy a **Hosted Agent** to Microsoft Foundry using the Azure Developer CLI (`azd`) and the official starter template.

## Why a Dedicated Spoke?

> ⚠️ **Important**: Hosted Agents currently **do not support** consuming LLMs via API Management (APIM) gateway connections.

Unlike prompt-based agents (Lab 1B), hosted agents require **direct model deployments** within the Foundry account. This means:

| Scenario | Gateway (APIM) Support | Model Location |
|----------|------------------------|----------------|
| Prompt Agents | ✅ Yes | Central Landing Zone via APIM |
| **Hosted Agents** | ❌ No | Must deploy models in same account |

This lab deploys a **dedicated spoke** - a team's own Foundry Account with:
- Its own model deployments (can't share via gateway)
- Container Registry for agent images
- Capability Host for running hosted agents
- Application Insights for observability

## Approach: Official Starter Template

This lab uses the official `Azure-Samples/ai-foundry-starter-basic` template which:
- Provides production-ready Bicep infrastructure
- Auto-installs the `azd ai agent` extension
- Handles all RBAC and networking configuration

## Step 1: Create Project Directory

Create a fresh directory for the hosted agent project.

In [1]:
import os

# Create and change to project directory
PROJECT_DIR = "/workspaces/getting-started-with-foundry/hosted-agent-project"
os.makedirs(PROJECT_DIR, exist_ok=True)
os.chdir(PROJECT_DIR)

print(f"✅ Working directory: {PROJECT_DIR}")

✅ Working directory: /workspaces/getting-started-with-foundry/hosted-agent-project


## Step 2: Initialize from Starter Template

Use `azd init` with the official AI Foundry starter template.

> **Note**: Hosted agents are only available in **North Central US**.

In [2]:
ENV_NAME = "hosted-agent-spoke"
LOCATION = "northcentralus"  # Required for hosted agents

# Initialize from the official starter template
!azd init --template Azure-Samples/ai-foundry-starter-basic --environment "{ENV_NAME}" --location "{LOCATION}" --no-prompt

print(f"\n✅ Project initialized with environment: {ENV_NAME}")


Initializing an app to run on Azure (azd init)

  (✓) Done: Initialized git repository/workspaces/getting-started-wi...
  (✓) Done: Downloading template code to: /workspaces/getting-started-with-foundry/hosted-agent-project


Installing required extensions...
  (-) Skipped: Installing azure.ai.agents extension (version 0.1.5-preview already installed)

SUCCESS: New project initialized!
You can view the template code in your directory: /workspaces/getting-started-with-foundry/hosted-agent-project
Learn more about running 3rd party code on our DevHub: https://aka.ms/azd-third-party-code-notice

✅ Project initialized with environment: hosted-agent-spoke


In [3]:
# View the generated project structure
!ls -la
!echo "\n--- azure.yaml ---"
!cat azure.yaml

total 48
drwx------ 14 vscode vscode   448 Jan 16 16:04 .
drwxr-xr-x 15 vscode vscode   480 Jan 16 16:04 ..
drwxr-xr-x  5 vscode vscode   160 Jan 16 16:04 .azure
drwxr-xr-x 10 vscode vscode   320 Jan 16 16:04 .git
drwxr-xr-x  5 vscode vscode   160 Jan 16 16:04 .github
-rw-r--r--  1 vscode vscode  7377 Jan 16 16:04 .gitignore
-rw-r--r--  1 vscode vscode   135 Jan 16 16:04 CHANGELOG.md
-rw-r--r--  1 vscode vscode  3959 Jan 16 16:04 CONTRIBUTING.md
-rw-r--r--  1 vscode vscode  1140 Jan 16 16:04 LICENSE.md
-rw-r--r--  1 vscode vscode 12967 Jan 16 16:04 README.md
-rw-r--r--  1 vscode vscode  2788 Jan 16 16:04 SECURITY.md
-rw-r--r--  1 vscode vscode   473 Jan 16 16:04 SUPPORT.md
-rw-r--r--  1 vscode vscode   339 Jan 16 16:04 azure.yaml
drwxr-xr-x  6 vscode vscode   192 Jan 16 16:04 infra
\n--- azure.yaml ---
# yaml-language-server: $schema=https://raw.githubusercontent.com/Azure/azure-dev/main/schemas/v1.0/azure.yaml.json
name: ai-foundry-starter-basic

infra:
  provider: bicep
  path: ./inf

## Step 3: Authenticate with Azure

Login to Azure and set the subscription.

In [ ]:
# Login to Azure (opens browser)
!azd auth login

In [ ]:
import subprocess

# Get subscription ID
SUB_ID = subprocess.run(
    'az account show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

print(f"Subscription: {SUB_ID}")

# Set default subscription for azd
!azd config set defaults.subscription "{SUB_ID}"

print(f"\n✅ Subscription configured")

## Step 4: Add Agent Definition

Initialize an agent from a sample or create your own.

The `azd ai agent init` command:
1. Downloads the agent definition into `src/`
2. Updates `azure.yaml` with service configuration
3. Maps environment variables

In [ ]:
# You may need to run this on your CLI directly

AGENT_URL = "https://github.com/microsoft-foundry/foundry-samples/blob/main/samples/python/hosted-agents/agent_framework/agents_in_workflow/agent.yaml"

!azd ai agent init -m "{AGENT_URL}" --no-prompt

print("\n✅ Agent initialized from sample")

In [ ]:
# View the updated project structure
!echo "--- Project Structure ---"
!find . -type f -name "*.yaml" -o -name "*.py" -o -name "Dockerfile" | head -20

!echo "\n--- azure.yaml (updated) ---"
!cat azure.yaml

In [ ]:
# View the agent definition
!echo "--- Agent Definition ---"
!cat src/*/agent.yaml 2>/dev/null || cat src/agent.yaml 2>/dev/null || echo "Agent yaml location varies by template"

## Step 5: Deploy Everything

⏱️ **Takes ~5-10 minutes**

The `azd up` command orchestrates:
1. **Provision infrastructure** - Creates AI Foundry account, project, ACR, model deployments
2. **Build container** - Builds and pushes agent image to ACR
3. **Deploy agent** - Creates hosted agent version and deployment

In [ ]:
# Deploy infrastructure + agent
!azd up --no-prompt

## Step 6: Get Deployment Outputs

In [ ]:
import subprocess

def get_azd_env(key):
    result = subprocess.run(f'azd env get-value {key}', shell=True, capture_output=True, text=True)
    return result.stdout.strip()

RESOURCE_GROUP = get_azd_env('AZURE_RESOURCE_GROUP')
PROJECT_ENDPOINT = get_azd_env('AZURE_AI_PROJECT_ENDPOINT')  # Standard key from starter template
PROJECT_NAME = get_azd_env('AZURE_AI_PROJECT_NAME')
ACCOUNT_NAME = get_azd_env('AZURE_AI_ACCOUNT_NAME')

print(f"Resource Group:    {RESOURCE_GROUP}")
print(f"Account Name:      {ACCOUNT_NAME}")
print(f"Project Name:      {PROJECT_NAME}")
print(f"Project Endpoint:  {PROJECT_ENDPOINT}")

## Step 7: Check Agent Status

In [22]:
# Alternative: Use az CLI to list agents
!az cognitiveservices agent list \
    --account-name "{ACCOUNT_NAME}" \
    --project-name "{PROJECT_NAME}" \
    -o table

Command group 'cognitiveservices agent' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Name                Object
------------------  --------
agents-in-workflow  agent


## Step 8: Test the Agent

Invoke the deployed agent using the Azure AI Projects SDK.

In [ ]:
!pip install azure-ai-projects==2.0.0b2 azure-identity -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from azure.ai.projects.models import AgentReference

# Get the first agent
agent = agents[0] if agents else None

if agent:
    # Invoke the agent
    openai_client = client.get_openai_client()
    
    response = openai_client.responses.create(
        input=[{"role": "user", "content": "Create a marketing strategy for an eco-friendly water bottle"}],
        extra_body={"agent": AgentReference(name=agent.name, version="1").as_dict()}
    )
    
    print("🤖 Agent Response:")
    print(response.output_text)
else:
    print("❌ No agents found. Check deployment status.")

## Done!

You've deployed a **Hosted Agent** using the official starter template.

### Key Takeaways

| Step | Command | Purpose |
|------|---------|--------|
| 1 | `azd init --template` | Scaffold from starter template |
| 2 | `azd ai agent init -m <url>` | Add agent from sample catalog |
| 3 | `azd up` | Provision + Build + Deploy |
| 4 | `azd ai agent status` | Check deployment status |

### Hosted Agents vs Prompt Agents

| Feature | Prompt Agents | Hosted Agents |
|---------|--------------|---------------|
| Gateway Support | ✅ APIM connection | ❌ Direct models only |
| Custom Code | ❌ Instructions only | ✅ Full container |
| Framework | Declarative YAML | Python/C# code |

---

## Cleanup

In [ ]:
# Delete everything (agent + infrastructure)
# !azd down --force --purge